In [4]:
import os

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

# os.environ.get('OPENAI_API_KEY')

True

In [5]:
from langchain_openai import ChatOpenAI

/Users/kariske/langchain/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [6]:
chat = ChatOpenAI(
    openai_api_key=os.environ.get('OPENAI_API_KEY'),
    openai_api_base=os.environ.get('CHATGPT_API_ENDPOINT')   
)

In [7]:
template = """
    回答如下問題，並準確說出計算結果
    問題：{query}
    AI:
"""

In [8]:
from langchain import PromptTemplate

In [9]:
prompt_template = PromptTemplate(
    template=template,
    input_variables=['query'],
)

In [10]:
from langchain.chains.llm import LLMChain

chain = LLMChain(
    llm=chat,
    prompt=prompt_template,
    verbose=True
)

/var/folders/1f/8yzdnxxs70zbqsthkv66wrvc0000gn/T/ipykernel_19590/2112724683.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(


In [11]:
# chain.run(query="3的三次方是多少?")
output = chain.invoke({"query": "18的0.6352次方是多少？"})
print(output['text'])



> Entering new LLMChain chain...
Prompt after formatting:

    回答如下問題，並準確說出計算結果
    問題：18的0.6352次方是多少？
    AI:


> Finished chain.
18的0.6352次方結果為4.699347。


In [12]:
from langchain.chains import LLMMathChain

math_chain = LLMMathChain.from_llm(
    llm=chat,
    verbose=True
)

In [13]:
result = math_chain.invoke({"question": "18的0.6352次方是多少?"})



> Entering new LLMMathChain chain...
18的0.6352次方是多少?```text
18**0.6352
```
...numexpr.evaluate("18**0.6352")...

Answer: 6.271177528595222
> Finished chain.


In [14]:
print(math_chain.prompt.template)

Translate a math problem into a expression that can be executed using Python's numexpr library. Use the output of running this code to answer the question.

Question: ${{Question with math problem.}}
```text
${{single line mathematical expression that solves the problem}}
```
...numexpr.evaluate(text)...
```output
${{Output of running the code}}
```
Answer: ${{Answer}}

Begin.

Question: What is 37593 * 67?
```text
37593 * 67
```
...numexpr.evaluate("37593 * 67")...
```output
2518731
```
Answer: 2518731

Question: 37593^(1/5)
```text
37593**(1/5)
```
...numexpr.evaluate("37593**(1/5)")...
```output
8.222831614237718
```
Answer: 8.222831614237718

Question: {question}



In [15]:
import re

def transform_func(inputs: dict):
    text = inputs['tr_text']
    text = re.sub(r'[ ]+', ' ', text)
    return {
        'tr_output': text
    }

In [16]:
from langchain.chains.transform import TransformChain

In [17]:
clean_space_chain = TransformChain(
    input_variables=['tr_text'],
    output_variables=['tr_output'],
    transform=transform_func
)

In [18]:
output = clean_space_chain.invoke({"tr_text": "           Hello            World!~"})
print(output['tr_output'])

 Hello World!~


In [33]:
template = """
    轉換如下文字:
    
    {tr_output}
    
    新形式{convert_style}
    
    轉換:
"""

In [34]:
prompt = PromptTemplate(
    template=template,
    # input_variables=['convert_output', 'convert_style']
    input_variables=['convert_style'],
)

In [35]:
convert_chain = LLMChain(
    llm=chat,
    prompt=prompt,
    output_key="new_output",
    verbose=True
)

In [ ]:
# new_output = convert_chain.invoke({
#     "convert_output": "Gong Hey Fat Choy",
#     "convert_style": "繁體中文"
# })
# print(new_output['new_output'])



> Entering new LLMChain chain...
Prompt after formatting:

    轉換如下文字:
    
    Gong Hey Fat Choy
    
    新形式繁體中文
    
    轉換:


> Finished chain.
恭喜發財


In [36]:
from langchain.chains import SequentialChain

In [38]:
sequential_chain = SequentialChain(
    chains=[clean_space_chain, convert_chain],
    input_variables=['tr_text', 'convert_style'],
    output_variables=['new_output'],
    verbose=True
)

In [40]:
input_text = """
"Gony Hey Fat Choy"           is a          traditional                       Cantonese phrase             used to convey """

In [42]:
ouput = sequential_chain.invoke({
    "tr_text": input_text,
    "convert_style": "繁體中文"
})
print(ouput['new_output'])



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

    轉換如下文字:
    
    
"Gony Hey Fat Choy" is a traditional Cantonese phrase used to convey 
    
    新形式繁體中文
    
    轉換:


> Finished chain.

> Finished chain.
"恭喜發財" 是一個傳統的廣東話短語，用來表達祝福。
